In [160]:
import os
import json

import numpy as np
import pandas as pd

print(os.path.abspath(os.path.curdir))

/Users/nicholsn/Repos/metasearch/crawler/transform


In [209]:
# Path to projects with extracted files.
extract_path = os.path.abspath('../extract')
# Path to where the tranformed projects will go.
xfm_path = os.path.abspath(os.path.curdir)
# Project dirs.
project_names = [i for i in os.listdir(extract_path) if '.' not in i]
extract_dirs = [os.path.join(extract_path, i) for i in project_names]
xfm_dirs = [os.path.join(xfm_path, i) for i in project_names]

In [224]:
def apply_mapping(mapping, df_list):
    df = pd.DataFrame()
    csv = pd.concat(df_list)
    for col, elem in mapping.iteritems():
        series = csv[col]
        if elem.get('type') == 'category':
            result = series.apply(lambda x: elem.get(str(x)))
        elif elem.get('type') == 'number':
            lambda x: True if x % 2 == 0 else False
            result = series.apply(lambda x: np.NaN if str(x) in elem.keys() else x)
        else:
            result = series
        df[elem.get('element')] = result
    return df

In [228]:
# Read a mapping.json file from each sub directory to process.
for project_name in project_names:
    df_list = list()
    mapping = dict()
    extract_dir = os.path.join(extract_path, project_name)
    xfm_dir = os.path.join(xfm_path, project_name)
    # All files must use the same data dictionary in a given directory.
    extract_files = [os.path.join(extract_dir, i) for i in os.listdir(extract_dir)]
    mapping_file = os.path.join(xfm_dir, 'mapping.json')
    if os.path.exists(mapping_file):
        with open(mapping_file, 'rb') as fi:
            mapping.update(json.load(fi))
        for extract_file in extract_files:
            df_list.append(pd.read_csv(extract_file))
        xfm = apply_mapping(mapping, df_list)
        pheno_file = ''.join([project_name, '_', 'phenotype.csv'])
        pheno_path = os.path.join(xfm_dir, pheno_file)
        xfm.to_csv(pheno_path, index=False)

In [83]:
p = '/Users/nicholsn/Repos/metasearch/crawler/extract/adhd200'

In [149]:
foo = pd.DataFrame()
bar = pd.read_csv('/Users/nicholsn/Repos/metasearch/crawler/extract/adhd200/Brown_TestRelease_phenotypic.csv')
bat = pd.read_csv('/Users/nicholsn/Repos/metasearch/crawler/extract/adhd200/KKI_phenotypic.csv')

In [212]:
ls /Users/nicholsn/Repos/metasearch/crawler/transform/abide_initiative/mapping.json


/Users/nicholsn/Repos/metasearch/crawler/transform/abide_initiative/mapping.json


In [152]:
bat

,ScanDir ID,Site,Gender,Age,Handedness,DX,Secondary Dx,ADHD Measure,ADHD Index,Inattentive,...,Performance IQ,Full2 IQ,Full4 IQ,Med Status,QC_Rest_1,QC_Rest_2,QC_Rest_3,QC_Rest_4,QC_Anatomical_1,QC_Anatomical_2
0,2371032,3,0,10.73,1,0,NaN,2,47,55,...,119,NaN,122,1,1,NaN,NaN,NaN,1,NaN
1,2026113,3,0,12.99,1,1,NaN,2,90,89,...,108,NaN,106,1,1,NaN,NaN,NaN,1,NaN
2,3434578,3,0,8.12,1,0,NaN,2,42,42,...,98,NaN,89,1,1,NaN,NaN,NaN,1,NaN
3,8628223,3,0,10.81,1,0,Simple phobia,2,42,49,...,86,NaN,97,1,1,NaN,NaN,NaN,1,NaN
4,1623716,3,0,12.65,1,1,NaN,2,87,90,...,88,NaN,89,1,1,NaN,NaN,NaN,1,NaN
5,1594156,3,1,12.87,1,0,Simple Phobia,2,54,55,...,104,NaN,108,1,1,NaN,NaN,NaN,1,NaN
6,2930625,3,1,9.97,1,0,NaN,2,40,40,...,127,NaN,106,1,1,NaN,NaN,NaN,1,NaN
7,3154996,3,1,11.65,1,3,NaN,2,64,74,...,98,NaN,88,1,1,NaN,NaN,NaN,1,NaN
8,3160561,3,1,11.95,1,1,NaN,2,71,62,...,108,NaN,99,1,1,NaN,NaN,NaN,1,NaN
9,4275075,3,0,10.50,1,3,NaN,2,87,75,...,100,NaN,94,1,1,NaN,NaN,NaN,1,NaN
